# Gör en total scraping där alla avdelningar och all historik är öppna

In [ ]:
# imports
#!apt install chromium-chromedriver
#!pip install selenium

import pandas as pd
import numpy as np 
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
import time
import logging
logging.basicConfig(filename='tot.log', filemode='w', format='%(name)s - %(message)s',level=logging.INFO)



In [ ]:
def getBana(driver_s, part1,avd, after=True):
    hack = '2' if avd=='7' else '1'
    end_string = '/div[1]/div[1]/div/div[2]/div[2]/div[1]/span'
    if after:
        end_string = '/div[1]/div[1]/div/div[2]/div[2]/div[1]/span'

    pth = part1+'div['+avd+']'+'/div/div/div['+hack+']' + end_string
    bana = driver_s.find_element_by_xpath(pth)
    return bana.text
    
def getLoppDist(driver_s, part1,avd, after=True):
    hack = '2' if avd=='7' else '1'
    end_string='/div[1]/div[1]/div/div[2]/div[2]/div[2]/span'
    if after:
        end_string='/div[1]/div[1]/div/div[2]/div[2]/div[2]/span'

    pth = part1+'div[' + avd + ']/div/div/div['+hack+']' + end_string

    d = driver_s.find_element_by_xpath(pth)
    lopp_dist = d.text.split()[0]
    start = d.text.split()[1]

    return lopp_dist.replace('M',''), start

def getPris(driver_s,part1,avd,after=True):
    hack = '2' if avd=='7' else '1'
    pth=part1+'div['+avd+']/div/div/div['+hack+']/div[1]/div[2]/div/span[3]'
                
    prisrad = driver_s.find_element_by_xpath(pth).text
    priser = prisrad.replace('Pris: ','')
    priser = priser.replace('.', '')    
    
    return int(priser.split(sep='-')[0])

In [ ]:
def getNamn(driver_s, part1,avd, rad, after=True):
    
    end_string = '/td[1]/span[2]/div/span[1]'
    if after:
        end_string = '/td[1]/span/div/span[1]'

    pth = part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + end_string
    
    driver_s.implicitly_wait(5)     # seconds
    namn = driver_s.find_element_by_xpath(pth)
    driver_s.implicitly_wait(5)     # seconds
    
    return namn.text


def getKön(driver_s, part1, avd, rad, after=True):
    end_string = '/td[1]/span[2]/div/span[2]'
    if after:
        end_string = '/td[1]/span/div/span[2]'

    pth = part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + end_string
    
    try:
        kön = driver_s.find_element_by_xpath(pth)
        return kön.text
    except:
        print('inget kön, ingen ålder')
        # logging.warning('både kön och ålder saknas för häst '+str(i))
        return ' '


def getÅlder(driver_s, part1,avd, rad, after=True):
    end_string = '/td[1]/span[2]/div/span[3]'
    if after:
        end_string = '/td[1]/span/div/span[3]'

    pth = part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + end_string
    
    try:
        ålder = driver_s.find_elements_by_xpath(pth)
    except:
        return 0

    if len(ålder) == 0:
        return 0

    return ålder[0].text


def getSpår(driver_s, part1, avd, rad, after=True):  # spår och distans
    sp = '5'  
    end_string = '/td['+sp+']'
    pth = part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + end_string
    sp_dist = driver_s.find_element_by_xpath(pth)
    txt = sp_dist.text.split()
    spår = None
    dist = None
    if len(txt) > 0:
        dist = txt[0]
    if len(txt) == 3:
        spår = txt[2]

    return spår, dist


def getKusk(driver_s,part1, avd, rad, after=True):
    end_string = '/td[2]' # + '/span' ?
    pth = part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + end_string
    kusk = driver_s.find_element_by_xpath(pth)
    return kusk.text

   
def getStreck(driver_s,part1, avd, rad, after=True):
    end_string = '/td[3]'
    pth = part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + end_string
    streck = driver_s.find_element_by_xpath(pth)
    return streck.text.replace('%','')

def getVodds(driver_s, part1,avd, rad, after=False):
    vo = '6'  #
    end_string = '/td['+vo+']'
    pth = part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + end_string
    vodds = driver_s.find_element_by_xpath(pth)

    return vodds.text.replace(',','.')

def getPodds(driver_s, part1, avd, rad,  after=False):
    # po = '6'  # blev fel  direkt efter omg
    po = '7'  #
    end_string = '/td['+po+']'
    pth = part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + end_string
    podds = driver_s.find_element_by_xpath(pth)

    return podds.text.replace(',','.')

def getKrPerStart(driver_s, part1, avd, rad,  after=False):
    kr = '4'
    end_string = '/td['+kr+']'
    pth = part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + end_string
    kr = driver_s.find_element_by_xpath(pth)

    return kr.text.replace(' ','')


In [ ]:
def getPlac(driver_r,avd,rad):
    pth = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[8]/div['+avd+']/div/div/table/tbody/tr['+rad+']/td[1]'
    pl = the_driver.find_element_by_xpath(pth) 

    return pl.text    


In [ ]:
def ber_hästrad(häst,till,hist=False):
    if hist :
        return häst*2+till
    return häst*2-1+till  

def starta_upp(url,gecko):
    # get the webdriver for driver_s
    driver_s = webdriver.Chrome(executable_path=gecko)
    driver_s.get(url)              # startlist
    driver_s.implicitly_wait(5)     # seconds
    driver_s.fullscreen_window()
    
    # get the webdrir for driver_r
    driver_r = webdriver.Chrome(executable_path=gecko)
    driver_r.get(url+'/resultat')
    driver_r.implicitly_wait(5)     # seconds
    driver_r.fullscreen_window()

    import os
    input('sätt startlist till "allt" och utöka_alla i settings.vi resultat plac och utdelning')
    return driver_s,driver_r


def en_history_rad(driver_s,dic,part1,part2,avd,rad,histrad,after):
    h='h'+str(histrad)+'_'
    
    dat=part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + part2 + 'tr['+histrad+']/td[1]'  # dat 
    ban=part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + part2 + 'tr['+histrad+']/td[2]'  # bana
    kus=part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + part2 + 'tr['+histrad+']/td[3]'  # kusk
    pla=part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + part2 + 'tr['+histrad+']/td[5]'  # plac
    dis=part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + part2 + 'tr['+histrad+']/td[6]'  # dist
    spå=part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + part2 + 'tr['+histrad+']/td[7]'  # spår
    kmt=part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + part2 + 'tr['+histrad+']/td[10]' # km-tid
    odd=part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + part2 + 'tr['+histrad+']/td[12]' # odds
    pri=part1 + 'div[' + avd + ']/div/div/table/tbody/tr[' + rad + ']' + part2 + 'tr['+histrad+']/td[13]' # pris
    
    try: # hx_dat
        driver_s.implicitly_wait(2)                             # seconds
        datum = driver_s.find_element_by_xpath(dat)
        driver_s.implicitly_wait(2)                             # seconds
        dic[h+'dat'].append(datum.text)
        logging.warning(f'datum={datum.text} avd={avd} radix={rad} histrad={histrad}')
    except:
        print(f"##### hist except på datum ###### avd={avd}, radix={rad}, histrad={histrad}")
        logging.warning(f'dat: {dat}')
        dic[h+'dat'].append(None)
        dic[h+'bana'].append(None)
        dic[h+'kusk'].append(None)
        dic[h+'plac'].append(None)
        dic[h+'spår'].append(None)
        dic[h+'kmtid'].append(None)
        dic[h+'odds'].append(None)
        dic[h+'pris'].append(None)
    
        return dic

    bana = driver_s.find_element_by_xpath(ban).text
    dic[h+'bana'].append(bana.split('-')[0])
    dic[h+'kusk'].append(driver_s.find_element_by_xpath(kus).text)
    dic[h+'plac'].append(driver_s.find_element_by_xpath(pla).text)
    dic[h+'spår'].append(driver_s.find_element_by_xpath(spå).text)
    dic[h+'kmtid'].append(driver_s.find_element_by_xpath(kmt).text)
    dic[h+'odds'].append(driver_s.find_element_by_xpath(odd).text.replace(',','.'))
    dic[h+'pris'].append(driver_s.find_element_by_xpath(pri).text.replace("'",""))
        
    return dic

def en_history_grupp(driver_s,dic,part1,avd,rad,after):
    part2 = '/td/div[2]/table/tbody/'  # for history
    for histrad in [1,2,3,4,5]:
        dic=en_history_rad(driver_s,dic,part1,part2,str(avd),str(rad),str(histrad),after)
    return dic

def en_häst_rad(driver_s,dic,part1,rad,datum,avd ,bana,lopp_dist,start,plac,pris,after=True):
    # print(f'part1={part1} avd={avd} rad={rad} after={after}')

    dic['datum'].append(datum)
    dic['avd'].append(avd)
    dic['bana'].append(bana)
    dic['lopp_dist'].append(lopp_dist)
    dic['plac'].append(plac)
    dic['pris'].append(pris)
    dic['start'].append(start)

    dic['häst'].append(getNamn(driver_s,part1,avd,rad,after))
    dic['kön'].append(getKön(driver_s,part1,avd,rad,after))
    dic['ålder'].append(getÅlder(driver_s,part1,avd,rad,after))
    dic['kusk'].append(getKusk(driver_s,part1,avd,rad,after))
    dic['streck'].append(getStreck(driver_s,part1,avd,rad,after))
    dic['vodds'].append(getVodds(driver_s,part1,avd,rad,after))
    dic['podds'].append(getPodds(driver_s,part1,avd,rad,after))
    spår,dist = getSpår(driver_s,part1,avd,rad,after)
    dic['spår'].append(spår)
    dic['dist'].append(dist)
    dic['kr'].append(getKrPerStart(driver_s, part1, avd, rad, after))

    return dic

def plac_start(driver_r,avd):
    def getStartnr(text):
        nr=text.strip()[:2]
        try:
            nr=int(nr)
        except:
            try:
                nr = int(nr[0])
            except:
                pass           
        return nr  

    pth='//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[7]/div['+avd+']/div/div/table/tbody'
    el=driver_r.find_elements_by_xpath(pth)
    # print(el[0].text)
    el =el[0].text.splitlines()
    sdic={}
    for e in el:
        plac = e.strip()[:2]
        startnr=e[2:]
        if plac=='ut':
            plac='u'
            startnr = e[4:] 
        startnr = getStartnr(startnr)    
        sdic[startnr]=plac
    return sdic
    
def en_avdelning(driver_s,driver_r,part1,datum,avd,bana,dic,after):  # ett lopp
    
    # för att koppla startlista til placering
    sdic = plac_start(driver_r,avd)
    # print(sdic)
    pris=getPris(driver_s,part1,avd,after)
    lopp_dist,start = getLoppDist(driver_s,part1,avd,after)
    avd_pth = part1 + 'div[' + avd + ']/div/div/table/tbody'
    avd_tot = driver_s.find_elements_by_xpath(avd_pth)
    lista = avd_tot[0].text.split('SENASTE ')  # ger en grupp per häst
    logging.warning(datum+' avd: '+str(avd))
    till=0
    for nr,texter in enumerate(lista[:-1]):
        gr = texter.splitlines()
        if len(gr)>2 and gr[-3].startswith('Tillägg:'):
            print('tillägg funnet före',nr+1)
            till+=1
        print(f'börja med {datum} avd={avd} nr={nr+1}')
        rad = ber_hästrad(nr+1,till,hist=False)
        dic=en_häst_rad(driver_s,dic,part1,str(rad),datum,avd, bana,lopp_dist,start,sdic[nr+1],pris,after)
        rad = ber_hästrad(nr+1,till,hist=True)
        dic=en_history_grupp(driver_s,dic,part1,avd,str(rad),after)
    return dic

def en_vecka(driver_s,driver_r,datum,dic,after): # en omgång
       
    bana = getBana(driver_s,part1,'1',after)
    for avd in [1,2,3,4,5,6,7]:
        dic = en_avdelning(driver_s,driver_r,part1,datum,str(avd),bana,dic,after)
    return dic

def  skapa_df(dic):
    df=pd.DataFrame(dic)
    return df

def main(driver_s,driver_r,part1,dic,after):
    
    omgångar=pd.read_csv('omg_to_learn_link.csv')
    for cnt,url in enumerate(omgångar.links):
        datum = url.split('spel/')[1][0:10]  #gräv ut datum  
        logging.warning(f'{cnt} {datum}')
        print('omgång',datum)
        driver_s.get(url)              # startlist
        driver_s.implicitly_wait(5)     # seconds
        driver_s.fullscreen_window()
        driver_r.get(url+'/resultat')   # reultat
        driver_r.implicitly_wait(5)     # seconds
        driver_r.fullscreen_window()
    
        dic=en_vecka(driver_s,driver_r,datum,dic,after)
        pd.DataFrame(dic).to_csv('scrape_del3.csv',index=False,date_format='%s')
    print(len(dic))
    df=skapa_df(dic)
    print(df.shape)
    return df


## Starta main här

In [ ]:
url=f'https://www.atg.se/spel/2021-03-06/V75/aby'
gecko = 'C:\\Users\peter\\Documents\\MyProjects\\gecko\\chromedriver.exe'
part1 = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[8]/'
driver_s, driver_r = starta_upp(url,gecko)


In [ ]:
dic={'datum':[],'avd':[],'bana':[],'häst':[],'kusk':[], 'streck':[],'vodds':[],'podds':[],'kr':[],'spår':[],
     'dist':[], 'lopp_dist':[],'start':[],'ålder':[],'kön':[],'plac':[],'pris':[],
     'h1_dat':[],'h1_kusk':[],'h1_bana':[],'h1_spår':[],'h1_plac':[],'h1_pris':[],'h1_odds':[], 'h1_kmtid':[],
     'h2_dat':[],'h2_kusk':[],'h2_bana':[],'h2_spår':[],'h2_plac':[],'h2_pris':[],'h2_odds':[],'h2_kmtid':[],
     'h3_dat':[],'h3_kusk':[],'h3_bana':[],'h3_spår':[],'h3_plac':[],'h3_pris':[],'h3_odds':[],'h3_kmtid':[],
     'h4_dat':[],'h4_kusk':[],'h4_bana':[],'h4_spår':[],'h4_plac':[],'h4_pris':[],'h4_odds':[],'h4_kmtid':[],
     'h5_dat':[],'h5_kusk':[],'h5_bana':[],'h5_spår':[],'h5_plac':[],'h5_pris':[],'h5_odds':[],'h5_kmtid':[],
     }
part1 = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[8]/'
part2 = '/td/div[2]/table/tbody/'  # for history

df=main(driver_s,driver_r,part1,dic,True)

In [ ]:
# close drivers
driver_s.quit()
driver_r.quit()

In [ ]:
# Slå ihop delresultaten och spara totalen
df2=pd.read_csv('scrape_del2.csv')
df1=pd.read_csv('scrape_del.csv')
df_tot = pd.concat([df1,df2,df])
df_tot.to_csv('all_scraped.csv',index=False,date_format='%s')

In [ ]:
df_tot.info()

# Testbädd

In [ ]:
part1 = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[8]/'
part2 = '/td/div[2]/table/tbody/'  # for history
dic={'datum':[],'avd':[],'bana':[],'häst':[],'kusk':[], 'streck':[],'vodds':[],'podds':[],'kr':[],'spår':[],
     'dist':[], 'lopp_dist':[],'start':[],'ålder':[],'kön':[],'plac':[],'pris':[],
     'h1_dat':[],'h1_kusk':[],'h1_bana':[],'h1_spår':[],'h1_plac':[],'h1_pris':[],'h1_odds':[], 'h1_kmtid':[],
     'h2_dat':[],'h2_kusk':[],'h2_bana':[],'h2_spår':[],'h2_plac':[],'h2_pris':[],'h2_odds':[],'h2_kmtid':[],
     'h3_dat':[],'h3_kusk':[],'h3_bana':[],'h3_spår':[],'h3_plac':[],'h3_pris':[],'h3_odds':[],'h3_kmtid':[],
     'h4_dat':[],'h4_kusk':[],'h4_bana':[],'h4_spår':[],'h4_plac':[],'h4_pris':[],'h4_odds':[],'h4_kmtid':[],
     'h5_dat':[],'h5_kusk':[],'h5_bana':[],'h5_spår':[],'h5_plac':[],'h5_pris':[],'h5_odds':[],'h5_kmtid':[],
     }
    
after=True
avd = 7;häst=1; histrad=1
datum = url.split('spel/')[1][0:10]  #gräv ut datum  
   
rad=ber_hästrad(1,0)   
en_vecka(url,driver_s,driver_r,datum,dic,after) # en omgång
scrape_df = pd.DataFrame(dic)
print(scrape_df.info())

# print(getPris(driver_s,part1,str(avd),str(rad)))

# print(getÅlder(driver_s,part1,str(avd),str(rad)) )  
# print(getKön(driver_s,part1,str(avd),str(rad)) ) 

# bana = getBana(driver_s, part1,str(avd), after)
# lopp_dist,start = getLoppDist(driver_s, part1,str(avd), after)
# print(f'bana={bana}, lopp_dist={lopp_dist}, start={start}')
# en_avdelning(driver_s,driver_r,part1,datum,str(avd),bana,dic,after)

# # history avd1 häst1
# avd=1;häst=1; histrad=1
# rad=ber_hästrad(häst,hist=False)
# print('\navd',avd,'häst',häst,'rad',rad,'histrad',histrad)
# en_häst_rad(driver_s,dic,part1,str(avd),str(rad),after)
# rad=ber_hästrad(häst,hist=True)
# en_history_rad(driver_s,dic,part1,part2,str(avd),str(rad),str(histrad),after)


In [ ]:
# scrape_df = pd.DataFrame(dic)
# pd.set_option('display.width',100)
# print(scrape_df[scrape_df.avd=='6'][[ 'häst','kön', 'plac', 'ålder', 'h1_dat','h1_plac','h1_pris','h5_dat','h5_plac','h5_spår']].head(20))

avd='1';after=True

sdic = plac_start(driver_r,avd)
    
pris=getPris(driver_s,part1,avd,after)
# lopp_dist,start = getLoppDist(driver_s,part1,avd,after)
avd_pth = part1 + 'div[' + avd + ']/div/div/table/tbody'
avd_tot = driver_s.find_elements_by_xpath(avd_pth)
lista = avd_tot[0].text.split('SENASTE ')  # ger en grupp per häst
# logging.warning(datum+' avd: '+str(avd))
till=0
for nr,texter in enumerate(lista[:-1]):
    gr = texter.splitlines()
    print('\n',gr)
    print(sdic[nr+1])

# pth='//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[8]/div[2]/div/div/table/tbody/tr[10]/td[1]/span/div/span[1]'
# namn=driver_s.find_element_by_xpath(pth)

In [ ]:
def getStartnr(text):
        nr=text.strip()[:2]
        try:
            nr=int(nr)
        except:
            try:
                nr = int(nr[0])
            except:
                pass    
        print(nr)           
        return nr  

pth='//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[7]/div['+avd+']/div/div/table/tbody'
el=driver_r.find_elements_by_xpath(pth)
# print(el[0].text)
el =el[0].text.splitlines()
print(el)
sdic={}
for e in el:
    plac = e[:2]
    print('pl',plac)
    startnr=e[2:]
    if plac=='ut':
        startnr = e[4:] 
    startnr = getStartnr(startnr)    
    sdic[startnr]=plac
sdic

In [ ]:
' dddddd '.strip()